In [3]:
import torch
import scipy.sparse as sp
import numpy as np
import os
from NeuralNetwork import DeepMolH
import matplotlib.pyplot as plt
import scipy.sparse as sp
# 假设你的模型类在这个文件中

class HamiltonianPredictor:
    def __init__(self, model_path):
        self.model_path = model_path
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = self.load_model()
    
    def load_model(self):
        """加载训练好的模型"""
        print(f"Loading model from {self.model_path}...")
        model = DeepMolH().to(self.device)
        
        checkpoint = torch.load(self.model_path, map_location=self.device)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()  # 设置为评估模式
        
        print("Model loaded successfully!")
        return model
    
    def predict_hamiltonian(self, mol2_path):
        """预测单个分子的哈密顿量"""
        print(f"Predicting Hamiltonian for {mol2_path}...")
        
        with torch.no_grad():
            H_pred = self.model(mol2_path)
        
        H_pred_np = H_pred.cpu().numpy()
        print(f"Predicted Hamiltonian shape: {H_pred_np.shape}")
        
        return H_pred_np
    
    def predict_batch(self, mol2_paths):
        """批量预测多个分子的哈密顿量"""
        results = {}
        for mol_path in mol2_paths:
            if os.path.exists(mol_path):
                H_pred = self.predict_hamiltonian(mol_path)
                mol_name = os.path.basename(mol_path).split('.')[0]
                results[mol_name] = H_pred
            else:
                print(f"Warning: {mol_path} not found!")
        
        return results
    
    def save_hamiltonian(self, H_matrix, output_path, format='npz'):
        """保存哈密顿量矩阵"""
        if format == 'npz':
            # 保存为稀疏矩阵以节省空间
            H_sparse = sp.csr_matrix(H_matrix)
            sp.save_npz(output_path, H_sparse)
        elif format == 'npy':
            np.save(output_path, H_matrix)
        else:
            raise ValueError("Format must be 'npz' or 'npy'")
        
        print(f"Hamiltonian saved to {output_path}")

# 使用示例
def main():
    # 初始化预测器
    model_path = '/Users/jiaoyuan/Documents/GitHub/DeepMolH/DeepMolH/best_deepmolh_model.pth'
    predictor = HamiltonianPredictor(model_path)
    
    # 单个分子预测
    mol2_path = '/Users/jiaoyuan/Documents/GitHub/DeepMolH/DeepMolH/dataset/mol/60.mol2'
    H_pred = predictor.predict_hamiltonian(mol2_path)
    
    # 保存结果
    plt.imshow(H_pred)
    plt.colorbar()
    plt.title("Predicted Hamiltonian Matrix")
    plt.show()
    
    H_target = sp.load_npz('/Users/jiaoyuan/Documents/GitHub/DeepMolH/DeepMolH/dataset/Hamiltonian/60_h.npz')
    
    plt.imshow(H_target.toarray())
    plt.colorbar()
    plt.title("Target Hamiltonian Matrix")
    plt.show()
    
    return H_pred

if __name__ == '__main__':
    main()

Loading model from /Users/jiaoyuan/Documents/GitHub/DeepMolH/DeepMolH/best_deepmolh_model.pth...


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


RuntimeError: Error(s) in loading state_dict for DeepMolH:
	Unexpected key(s) in state_dict: "nodes.nodes_embedding.0.weight", "nodes.nodes_embedding.0.bias", "nodes.nodes_embedding.2.weight", "nodes.nodes_embedding.2.bias", "nodes.nodes_embedding.4.weight", "nodes.nodes_embedding.4.bias", "nodes.nodes_embedding.6.weight", "nodes.nodes_embedding.6.bias". 